In [1]:
from theano.sandbox import cuda

Using cuDNN version 5103 on context None
Preallocating 10867/11439 Mb (0.950000) on cuda
Mapped name None to device cuda: Tesla K40c (0000:81:00.0)


SkipTest: You are importing theano.sandbox.cuda. This is the old GPU back-end and is removed from Theano. Use Theano 0.9 to use it. Even better, transition to the new GPU back-end! See https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
model_path = 'data/imdb/models/'

# Data

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [5]:
idx_arr = sorted(idx,key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [6]:
idx2word = {v: k for k,v in idx.iteritems()} #iteritems returns iterator over dict's key/value pairs

In [7]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
len(x_train)
#print(x_train[0][:3])

25000

In [9]:
', '.join(map(str, x_train[0])) #join word IDs in x_train[0] with commas

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [10]:
idx2word[23022] #looks up word ID in dict and gives corresponding value k in word index dict

'bromwell'

In [11]:
' '.join(idx2word[o] for o in x_train[0])

"bromwell high is a cartoon comedy it ran at the same time as some other programs about school life such as teachers my 35 years in the teaching profession lead me to believe that bromwell high's satire is much closer to reality than is teachers the scramble to survive financially the insightful students who can see right through their pathetic teachers' pomp the pettiness of the whole situation all remind me of the schools i knew and their students when i saw the episode in which a student repeatedly tried to burn down the school i immediately recalled at high a classic line inspector i'm here to sack one of your teachers student welcome to bromwell high i expect that many adults of my age think that bromwell high is far fetched what a pity that it isn't"

1 is positive labels, 0 are negative

In [12]:
labels_train[:10] #this came from code copied from keras.datasets

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [13]:
#limit the size of vocab to 5000
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [14]:
lens = np.array(map(len,trn)) #create array of lengths of sentences in words
(lens.max(),lens.min(),lens.mean())

(2493, 10, 237.71364)

In [15]:
#pad w/ 0s or truncate to make all the same length
seq_len = 500

#use keras pad_sequences
trn = sequence.pad_sequences(trn, maxlen=seq_len,value=0)
test = sequence.pad_sequences(test,maxlen=seq_len,value=0)

In [16]:
trn.shape

(25000, 500)

# Models

### Single hidden layer NN

In [17]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])
#in embedding layer 32 is its output dimension

In [18]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 500, 32)       160000      embedding_input_1[0][0]          
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 16000)         0           embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 100)           1600100     flatten_1[0][0]                  
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 100)           0           dense_1[0][0]                    
___________________________________________________________________________________________

In [19]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 1s - loss: 0.4633 - acc: 0.7530 - val_loss: 0.2903 - val_acc: 0.8755
Epoch 2/2
25000/25000 [==============================] - 1s - loss: 0.1978 - acc: 0.9265 - val_loss: 0.3002 - val_acc: 0.8736


### Single Conv Layer, max pooling

In [20]:
conv1 = Sequential([
    Embedding(vocab_size,32,input_length=seq_len,dropout=0.2),
    Dropout(0.2),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.2),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [21]:
conv1.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [22]:
conv1.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 6s - loss: 0.5287 - acc: 0.7035 - val_loss: 0.3036 - val_acc: 0.8762
Epoch 2/4
25000/25000 [==============================] - 6s - loss: 0.2980 - acc: 0.8815 - val_loss: 0.2757 - val_acc: 0.8865
Epoch 3/4
25000/25000 [==============================] - 6s - loss: 0.2626 - acc: 0.8968 - val_loss: 0.2599 - val_acc: 0.8940
Epoch 4/4
25000/25000 [==============================] - 6s - loss: 0.2398 - acc: 0.9072 - val_loss: 0.2735 - val_acc: 0.8850


In [23]:
conv1.save_weights(model_path + 'conv1.h5')

In [24]:
conv1.load_weights(model_path + 'conv1.h5')

### Pre-trained vectors

In [25]:
def get_glove_dataset(dataset):
    md5sums = {'6B.50d': '8e1557d1228decbda7db6dfd81cd9909',
               '6B.100d': 'c92dbbeacde2b0384a43014885a60b2c',
               '6B.200d': 'af271b46c04b0b2e41a84d8cd806178d',
               '6B.300d': '30290210376887dcc6d0a5a6374d8255'}
    glove_path = os.path.abspath('data/glove/results')
    %mkdir -p $glove_path
    return get_file(dataset,
                    'http://files.fast.ai/models/glove/' + dataset + '.tgz',
                    cache_subdir=glove_path,
                    md5_hash=md5sums.get(dataset, None),
                    untar=True)

In [26]:
def load_vectors(loc):
    return (load_array(loc+'.dat'),
            pickle.load(open(loc+'_words.pkl','rb')),
            pickle.load(open(loc+'_idx.pkl','rb')))

In [27]:
vecs,words,wordidx = load_vectors(get_glove_dataset('6B.50d'))

Untaring file...


In [28]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((vocab_size,n_fact))
    
    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx[word]
            emb[i] = vecs[src_idx]
        else:
            emb[i] = normal(scale=0.6, size=(n_fact,))
            
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [29]:
emb = create_emb()

In [30]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=False),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [31]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [32]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 5s - loss: 0.5928 - acc: 0.6755 - val_loss: 0.4977 - val_acc: 0.7888
Epoch 2/2
25000/25000 [==============================] - 5s - loss: 0.4962 - acc: 0.7677 - val_loss: 0.4408 - val_acc: 0.8086


In [33]:
model.layers[0].trainable=True

In [34]:
model.optimizer.lr=1e-4

In [35]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=1,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 5s - loss: 0.4743 - acc: 0.7800 - val_loss: 0.4220 - val_acc: 0.8173


In [36]:
model.save_weights(model_path+'glove50.h5')

### Multi-size CNN

In [37]:
from keras.layers import Merge

In [38]:
graph_in = Input((vocab_size,50))
convs=[ ]
for fsz in range(3,6):
    x = Convolution1D(64,fsz,border_mode='same',activation='relu')(graph_in)
    x=MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x)
out = Merge(mode="concat")(convs)
graph = Model(graph_in,out)

In [39]:
emb = create_emb()

In [40]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.2),
    graph,
    Dropout(0.5),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="sigmoid")
])

In [41]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [42]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.4922 - acc: 0.7460 - val_loss: 0.3446 - val_acc: 0.8453
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.3153 - acc: 0.8713 - val_loss: 0.2588 - val_acc: 0.8945


In [43]:
model.layers[0].trainable=False

In [44]:
model.optimizer.lr=1e-5

In [45]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.2821 - acc: 0.8868 - val_loss: 0.2619 - val_acc: 0.8956
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2621 - acc: 0.8953 - val_loss: 0.2523 - val_acc: 0.8979


### My own CNN

We'll use the pre-trained weights here.

In [46]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=False),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.35),
    MaxPooling1D(),
    Convolution1D(32,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [47]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [48]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 9s - loss: 0.6021 - acc: 0.6636 - val_loss: 0.5356 - val_acc: 0.7488
Epoch 2/2
25000/25000 [==============================] - 9s - loss: 0.4895 - acc: 0.7690 - val_loss: 0.4790 - val_acc: 0.8149


In [49]:
model.layers[0].trainable=False

In [50]:
model.optimizer.lr=1e-5

In [51]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 9s - loss: 0.4668 - acc: 0.7838 - val_loss: 0.4966 - val_acc: 0.7844
Epoch 2/2
25000/25000 [==============================] - 9s - loss: 0.4520 - acc: 0.7905 - val_loss: 0.4271 - val_acc: 0.8276


### Smaller CNN

In [52]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=False),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Convolution1D(32,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [53]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [54]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s - loss: 0.6263 - acc: 0.6241 - val_loss: 0.5183 - val_acc: 0.7902
Epoch 2/2
25000/25000 [==============================] - 7s - loss: 0.4965 - acc: 0.7677 - val_loss: 0.4788 - val_acc: 0.8051


In [55]:
model.layers[0].trainable=False

In [56]:
model.optimizer.lr=1e-5

In [57]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 7s - loss: 0.4704 - acc: 0.7826 - val_loss: 0.4482 - val_acc: 0.8048
Epoch 2/2
25000/25000 [==============================] - 7s - loss: 0.4576 - acc: 0.7887 - val_loss: 0.4231 - val_acc: 0.8354


### another one

In [58]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=True),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.35),
    MaxPooling1D(),
    Convolution1D(32,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [59]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [60]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.5592 - acc: 0.6663 - val_loss: 0.4817 - val_acc: 0.8406
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.3274 - acc: 0.8658 - val_loss: 0.3244 - val_acc: 0.8970


In [61]:
model.optimizer.lr=1e-5

In [63]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=5,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 11s - loss: 0.2485 - acc: 0.8991 - val_loss: 0.3436 - val_acc: 0.8559
Epoch 2/5
25000/25000 [==============================] - 11s - loss: 0.2418 - acc: 0.9036 - val_loss: 0.2725 - val_acc: 0.9017
Epoch 3/5
25000/25000 [==============================] - 11s - loss: 0.2296 - acc: 0.9109 - val_loss: 0.2765 - val_acc: 0.9019
Epoch 4/5
25000/25000 [==============================] - 11s - loss: 0.2238 - acc: 0.9111 - val_loss: 0.2930 - val_acc: 0.8928
Epoch 5/5
25000/25000 [==============================] - 11s - loss: 0.2179 - acc: 0.9149 - val_loss: 0.2607 - val_acc: 0.9027


In [64]:
model.optimizer.lr=1e-4

In [69]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.1865 - acc: 0.9254 - val_loss: 0.2512 - val_acc: 0.9035
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.1820 - acc: 0.9298 - val_loss: 0.2449 - val_acc: 0.9044


In [70]:
model.layers[0].trainable=False

In [71]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 11s - loss: 0.1788 - acc: 0.9319 - val_loss: 0.2532 - val_acc: 0.9012
Epoch 2/2
25000/25000 [==============================] - 11s - loss: 0.1708 - acc: 0.9350 - val_loss: 0.2474 - val_acc: 0.9028


In [73]:
from keras.optimizers import Nadam
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [74]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 9s - loss: 0.1770 - acc: 0.9316 - val_loss: 0.2488 - val_acc: 0.9016
Epoch 2/4
25000/25000 [==============================] - 9s - loss: 0.1760 - acc: 0.9324 - val_loss: 0.2718 - val_acc: 0.8885
Epoch 3/4
25000/25000 [==============================] - 9s - loss: 0.1688 - acc: 0.9348 - val_loss: 0.2492 - val_acc: 0.9026
Epoch 4/4
25000/25000 [==============================] - 9s - loss: 0.1693 - acc: 0.9357 - val_loss: 0.2834 - val_acc: 0.8820


### can we not overfit pls

In [75]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=True),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.35),
    MaxPooling1D(),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid')
])

In [76]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [77]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 10s - loss: 0.4898 - acc: 0.7376 - val_loss: 0.3099 - val_acc: 0.8863
Epoch 2/4
25000/25000 [==============================] - 10s - loss: 0.3062 - acc: 0.8768 - val_loss: 0.3554 - val_acc: 0.8439
Epoch 3/4
25000/25000 [==============================] - 10s - loss: 0.2749 - acc: 0.8894 - val_loss: 0.2672 - val_acc: 0.8978
Epoch 4/4
25000/25000 [==============================] - 10s - loss: 0.2581 - acc: 0.8983 - val_loss: 0.2917 - val_acc: 0.8934


### ditch the pretrained weights?

In [78]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.35),
    MaxPooling1D(),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid', W_regularizer=l2(0.01))
])

In [79]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [80]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 10s - loss: 0.6955 - acc: 0.4950 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/4
25000/25000 [==============================] - 10s - loss: 0.6932 - acc: 0.4957 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 3/4
25000/25000 [==============================] - 10s - loss: 0.6932 - acc: 0.5026 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/4
25000/25000 [==============================] - 10s - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6932 - val_acc: 0.5000


### they back but w reg this time

In [81]:
model = Sequential([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,
             weights=[emb],trainable=True),
    Dropout(0.25),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.35),
    MaxPooling1D(),
    Convolution1D(64,5,border_mode='same',activation='relu'),
    Dropout(0.5),
    MaxPooling1D(),
    Flatten(),
    Dense(100,activation='relu'),
    Dropout(0.7),
    Dense(1,activation='sigmoid', W_regularizer=l2(0.01))
])

In [82]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [83]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=4,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 10s - loss: 0.4590 - acc: 0.7740 - val_loss: 0.3399 - val_acc: 0.8926
Epoch 2/4
25000/25000 [==============================] - 10s - loss: 0.3064 - acc: 0.8811 - val_loss: 0.3116 - val_acc: 0.8973
Epoch 3/4
25000/25000 [==============================] - 10s - loss: 0.2718 - acc: 0.8901 - val_loss: 0.2779 - val_acc: 0.9027
Epoch 4/4
25000/25000 [==============================] - 10s - loss: 0.2539 - acc: 0.9021 - val_loss: 0.2864 - val_acc: 0.8934


we got em but sort of overfit on the last epoch maybe do less nxt time that was p good tho

### that multilayer thing WITH REGULARIZATION

In [84]:
graph_in = Input((vocab_size,50))
convs=[ ]
for fsz in range(3,6):
    x = Convolution1D(64,fsz,border_mode='same',activation='relu')(graph_in)
    x=MaxPooling1D()(x)
    x = Flatten()(x)
    convs.append(x)
out = Merge(mode="concat")(convs)
graph = Model(graph_in,out)

In [85]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.2),
    graph,
    Dropout(0.5),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="sigmoid", W_regularizer=l2(0.01))
])

In [86]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [92]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

RuntimeError: The model needs to be compiled before being used.

In [88]:
model.layers[0].trainable=False

In [89]:
model.optimizer.lr=1e-5

In [90]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.2858 - acc: 0.8884 - val_loss: 0.2637 - val_acc: 0.8941
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2621 - acc: 0.8984 - val_loss: 0.2599 - val_acc: 0.8966


### what if we do softmax instead?????

In [91]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.2),
    graph,
    Dropout(0.5),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="softmax", W_regularizer=l2(0.01))
])

In [93]:
model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])

In [94]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 7.9751 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000


ok that was terrible

In [98]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.2),
    graph,
    Dropout(0.4),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="sigmoid", W_regularizer=l2(0.01))
])

In [99]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [100]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.4014 - acc: 0.8279 - val_loss: 0.3219 - val_acc: 0.8706
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2974 - acc: 0.8813 - val_loss: 0.2753 - val_acc: 0.8895


In [101]:
model.optimizer.lr=1e-4

In [102]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.2680 - acc: 0.8926 - val_loss: 0.2601 - val_acc: 0.8960
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2480 - acc: 0.9006 - val_loss: 0.2555 - val_acc: 0.8992


wat if we do more dropout

In [103]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.2),
    graph,
    Dropout(0.5),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="sigmoid", W_regularizer=l2(0.01))
])

In [104]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [105]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.3963 - acc: 0.8267 - val_loss: 0.2992 - val_acc: 0.8816
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2961 - acc: 0.8817 - val_loss: 0.2673 - val_acc: 0.9002


In [106]:
model.optimizer.lr=1e-5

In [107]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.2682 - acc: 0.8926 - val_loss: 0.2655 - val_acc: 0.9014
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2475 - acc: 0.9015 - val_loss: 0.2581 - val_acc: 0.8984


WE BEAT HIM but still overfit @ the end saddddddd

In [108]:
model = Sequential ([
    Embedding(vocab_size,50,input_length=seq_len,dropout=0.2,weights=[emb]),
    Dropout(0.25),
    graph,
    Dropout(0.5),
    Dense(100,activation="relu"),
    Dropout(0.7),
    Dense(1,activation="sigmoid", W_regularizer=l2(0.01))
])

In [109]:
model.compile(loss='binary_crossentropy',optimizer=Nadam(),metrics=['accuracy'])

In [110]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.3986 - acc: 0.8272 - val_loss: 0.2938 - val_acc: 0.8948
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.3014 - acc: 0.8759 - val_loss: 0.2701 - val_acc: 0.8986


In [111]:
model.optimizer.lr=1e-5

In [112]:
model.layers[0].trainable=False

In [113]:
model.fit(trn,labels_train,validation_data=(test,labels_test),nb_epoch=2,batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 18s - loss: 0.2704 - acc: 0.8922 - val_loss: 0.2744 - val_acc: 0.8918
Epoch 2/2
25000/25000 [==============================] - 18s - loss: 0.2503 - acc: 0.9023 - val_loss: 0.2672 - val_acc: 0.8982
